In [6]:
# This code is from Qiskit Hackathon 2021 by the team
# Qiskit for high dimensional multipartite quantum states.
#
# Author: Hoang Van Do
#
# (C) Copyright 2021 Hoang Van Do, Tim Alexis Körner, Inho Choi, Timothé Presles and Élie Gouzien.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.

import numpy as np
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.exceptions import QiskitError

#Dephasing to m level. The inverse of the dephasing function would be giving the opposite phase Dephasing(m,-phase,dimension)
def singlephase(m, phase, dimension):
    if m > dimension:
        raise QiskitError('The level is higher than the dimension')
    n=int(np.ceil(np.log2(dimension)))
    qubits=QuantumRegister(n+1)
    circuit=QuantumCircuit(qubits)
    marray=[]
    for i in range(0,n): #bit decomposition
        if (( m >>  i) & 1) != 1 :
            marray.append(i)
    control_qubits=[]
    for i in range(0,n):
        control_qubits.append([i])
    target_qubit=[n]
    #check if m
    if len(marray)>0:
        circuit.x(marray)
        circuit.mcx(control_qubits,target_qubit)
        circuit.x(marray)
    for i in range(0,n):
        circuit.cp(phase,n,i)
    if len(marray)>0:
        #check if m, put back auxiliary qubit
        circuit.x(marray)
        circuit.mcx(control_qubits,target_qubit)
        circuit.x(marray)
    return circuit

from qiskit import Aer, execute
backend = Aer.get_backend('unitary_simulator')
np.set_printoptions(linewidth=200, precision=2, suppress=True)

qc = singlephase(2, np.pi/2, 8)
print(qc)
job = execute(qc, backend)
result = job.result()
U = result.get_unitary(qc)

N = int(U.shape[0]/2)
print("Auxiliary qubit should start and end in state |0> (only look at top left of matrix)")
print(U[:N,:N])


       ┌───┐     ┌───┐           ┌───┐                     ┌───┐
q60_0: ┤ X ├──■──┤ X ├─■─────────┤ X ├──────────────────■──┤ X ├
       └───┘  │  └───┘ │         └───┘                  │  └───┘
q60_1: ───────■────────┼────────■───────────────────────■───────
       ┌───┐  │  ┌───┐ │        │                ┌───┐  │  ┌───┐
q60_2: ┤ X ├──■──┤ X ├─┼────────┼────────■───────┤ X ├──■──┤ X ├
       └───┘┌─┴─┐└───┘ │P(π/2)  │P(π/2)  │P(π/2) └───┘┌─┴─┐└───┘
q60_3: ─────┤ X ├──────■────────■────────■────────────┤ X ├─────
            └───┘                                     └───┘     
Auxiliary qubit should start and end in state |0> (only look at top left of matrix)
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j